## 家計簿csvの処理の動作確認とか
- 一年間の月別支払い内容をcsvで読み込み
- ヘッダーの追加
- ４月以降のデータに修正（2025年度のみ）

In [27]:
import pandas as pd

csv_path = "/mnt/c/Users/paral/OneDrive/ドキュメント/enavi_household_2025.csv"

# ヘッダーなしで読み込む
df = pd.read_csv(csv_path, encoding="cp932", header=None)

# 列名を追加
df.columns = ["date", "purpose", "category", "payment_method", "amount"]

# 日付をdatetime型に変換
df["date"] = pd.to_datetime(df["date"], format="%Y/%m/%d")

# 4月以降のデータに絞る
df_april_onwards = df[df["date"].dt.month >= 4]

df_april_onwards.to_csv("/home/bodakko/kakeibo-app/backend/data/kakeibo_2025_april_onwards.csv", index=False, encoding="utf-8-sig")

# カテゴリ別合計
category_totals = df_april_onwards.groupby("category")["amount"].sum()

print(category_totals)
print(category_totals.sum())


category
ファッション     57958
交通        176670
交際費       104184
娯楽         22869
学習         18390
投資        207898
日用品       104316
未設定          385
水道・光熱      67596
美容         54100
贈り物        26858
通信         87650
食費         20545
Name: amount, dtype: int64
949419


## カテゴリ別合計

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rcParams

# 日本語フォントを明示的に指定
rcParams['font.family'] = 'Noto Sans CJK JP'

# カテゴリ別合計
category_totals = df_april_onwards.groupby("カテゴリ")["金額"].sum()

# グラフ作成
plt.figure(figsize=(10,6))
# カテゴリごとに色を割り当て
colors = plt.cm.tab10.colors  # 20色まで使えるカラーマップ
color_map = {cat: colors[i % len(colors)] for i, cat in enumerate(category_totals.index)}

category_totals.sort_values().plot(kind="barh", color=[color_map[cat] for cat in category_totals.index])
plt.title("カテゴリ別支出合計")
plt.xlabel("金額 (円)")
plt.ylabel("カテゴリ")
plt.grid(linestyle="-.", alpha=0.7)
plt.tight_layout()
plt.show()


## 変動費vs固定費

In [ ]:
# 固定費カテゴリ
fixed_categories = ["水道・光熱", "通信", "投資"]

# 固定費・変動費
fixed_total = df_april_onwards[df_april_onwards["カテゴリ"].isin(fixed_categories)]["金額"].sum()
variable_total = df_april_onwards[~df_april_onwards["カテゴリ"].isin(fixed_categories)]["金額"].sum()

# 全体合計
total = fixed_total + variable_total

print(f"全体合計: {total:,}円")
print(f"固定費合計: {fixed_total:,}円")
print(f"変動費合計: {variable_total:,}円")

# プロット
fig, ax = plt.subplots(figsize=(6,4))

# 落ち着いた色に設定
total_color = "#7f8c8d"       # グレー
variable_color = "#f39c12"    # オレンジ系
fixed_color = "#2980b9"       # 青系

# 左：全体合計
ax.bar(0, total, color=total_color, width=0.4, label='全体')
ax.text(0, total/2, f"{total:,}", ha='center', va='center', color='white', fontweight='bold')

# 右：固定費・変動費の積み上げ
ax.bar(1, variable_total, color=variable_color, width=0.4, label='変動費')
ax.bar(1, fixed_total, bottom=variable_total, color=fixed_color, width=0.4, label='固定費')

# 棒の中に値を表示
ax.text(1, variable_total/2, f"{variable_total:,}", ha='center', va='center', color='white', fontweight='bold')
ax.text(1, variable_total + fixed_total/2, f"{fixed_total:,}", ha='center', va='center', color='white', fontweight='bold')

# 軸の調整
ax.set_xticks([0,1])
ax.set_xticklabels(['合計', '内訳'])
ax.set_ylabel('金額')
ax.legend()

plt.tight_layout()
plt.show()

# 取引記録とカード履歴の結合

In [42]:
import pandas as pd

# -----------------------------
# 1. データ読み込み
# -----------------------------
# 取引記録CSV読み込み
# date, amount, type, method, category
transactions = pd.read_csv('/home/bodakko/kakeibo-app/backend/data/transaction_history.csv')
transactions['date'] = pd.to_datetime(transactions['date'])

# カード履歴CSV読み込み
# date, purpose, category, payment_method, amount
card_payments = pd.read_csv('/home/bodakko/kakeibo-app/backend/data/kakeibo_2025_april_onwards.csv')
card_payments['date'] = pd.to_datetime(card_payments['date'])

# -----------------------------
# 2. カード行を削除
# -----------------------------
transactions_card = transactions[transactions['category'] == 'カード'].copy()
transactions_no_card = transactions[transactions['category'] != 'カード'].copy()

# -----------------------------
# 3. 月列作成
# -----------------------------
transactions_card['month'] = transactions_card['date'].dt.to_period('M')
card_payments['month'] = card_payments['date'].dt.to_period('M')

# -----------------------------
# 4. カード明細を置き換え
# -----------------------------
added_rows = []

for idx, card_row in transactions_card.iterrows():
    month = card_row['month']
    # 同じ月のカード支払い明細を取得
    payments_in_month = card_payments[card_payments['month'] == month]
    
    for _, pay_row in payments_in_month.iterrows():
        # 元のカード行の情報をコピーして必要な列を上書き
        new_row = card_row.copy()
        new_row['date'] = pay_row['date']
        new_row['category'] = pay_row['category']
        new_row['amount'] = pay_row['amount']
        added_rows.append(new_row)

# DataFrame化
added_df = pd.DataFrame(added_rows)

# -----------------------------
# 5. 元の取引 + 追加分を結合
# -----------------------------
transactions_updated = pd.concat([transactions_no_card, added_df], ignore_index=True)

# 日付順にソート
transactions_updated = transactions_updated.sort_values('date').reset_index(drop=True)

# -----------------------------
# 6. amountを数値型に変換（任意）
# -----------------------------
transactions_updated['amount'] = (
    transactions_updated['amount']
    .astype(str)
    .str.replace('¥', '', regex=False)
    .str.replace(',', '', regex=False)
    .astype(float)
)

# -----------------------------
# 7. 結果確認
# -----------------------------
print(transactions_no_card.shape)
print(added_df['month'].value_counts())
print(transactions_updated.shape)

transactions_updated.to_csv(
    '/home/bodakko/kakeibo-app/backend/data/merged_transaction_history.csv',
    index=False,  # ← これを追加
    encoding='utf-8-sig'
)

(49, 9)
month
2025-04    40
2025-07    34
2025-10    32
2025-05    31
2025-08    26
2025-06    25
2025-09    23
Freq: M, Name: count, dtype: int64
(260, 9)
